In [102]:
from lib import data_loaders
from lib import dendra

from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [6]:
load_dotenv()
dendra_pass = os.getenv("DENDRA_PASS")

In [103]:
dendra.authenticate("dragar@lynker.com")

In [113]:
# parameters: start and end time
# Note the queries default to local time. Add a captial 'Z' to the end of the timestamp to indicate UTC and many functions have a parameter for local vs utc
begins_at = "1980-01-01T00:00:00"
ends_before = (
    dendra.time_format()
)  # time_format without argument gives current datetime. #'2020-03-01T00:00:00'

In [105]:
stations = dendra.list_stations("tnc")
for station in stations:
    print(station)

{'_id': '64078034a303bf7529cdb530', 'name': 'Cistern Water Tank', 'slug': 'cistern-water-tank'}
{'_id': '6390f04da9b6140fd21ce113', 'name': 'Cojo HQ Water Tank', 'slug': 'cojo-hq-water-tank'}
{'_id': '64dbe91cd0708704d6d53d8c', 'name': 'Dangermond Alegria', 'slug': 'dangermond-alegria'}
{'_id': '64dbe9ce2148db3f6f481b1a', 'name': 'Dangermond Alexander Ramajal', 'slug': 'dangermond-alexanderramajal'}
{'_id': '64dbe9c755264579ef3ea324', 'name': 'Dangermond Buckhorn 1', 'slug': 'dangermond-buckhorn1'}
{'_id': '64dbe9c780818732a0c9038b', 'name': 'Dangermond Buckhorn 2', 'slug': 'dangermond-buckhorn2'}
{'_id': '64dbe9c8d07087cecfd53d8e', 'name': 'Dangermond Cojo Canyon', 'slug': 'dangermond-cojocanyon'}
{'_id': '64dbe9c82148dbfa66481b12', 'name': 'Dangermond Damsite Canyon', 'slug': 'dangermond-damsitecanyon'}
{'_id': '64dbe9c880818761a2c9038d', 'name': 'Dangermond Diamond Corral', 'slug': 'dangermond-diamondcorral'}
{'_id': '64dbe9ce02096974eea07382', 'name': 'Dangermond East Ramajal', 'sl

In [112]:
measurement = "WellDepth"
query_refinement = {"is_hidden": False}
measurement_list = []  # list of only datastreams that you wish to download data from

ds_list = dendra.list_datastreams_by_measurement(
    measurement, "", [], "tnc", query_refinement
)

for ds in ds_list:
    dsm = dendra.get_meta_datastream_by_id(
        ds["_id"]
    )  # This will pull full datastream metadata in JSON format
    station_name = dsm["station_lookup"]["name"]
    print(station_name, ds["name"], ds["_id"])
    measurement_list.append(ds["_id"])

Dangermond Escondido 5 Depth to Groundwater 6511f67d2148db626248533e
Dangermond Buckhorn 2 Depth to Groundwater 6511f7e82148db3a1e485341
Dangermond Alegria Depth to Groundwater 6511f7e85526453a3c3edcd7
Dangermond Alexander Ramajal Depth to Groundwater 6511f7e8808187594cc93c0b
Dangermond Buckhorn 1 Depth to Groundwater 6511f7e8d070873fc4d574e2
Dangermond East Ramajal Depth to Groundwater 6511f7e90209692d22a0add0
Dangermond Diamond Corral Depth to Groundwater 6511f7e95526459d613edcd9
Dangermond Damsite Canyon Depth to Groundwater 6511f7e95c0d5fe521698416
Dangermond Cojo Canyon Depth to Groundwater 6511f7e9f1438dfeb44e11a9
Dangermond Escondido 3 Depth to Groundwater 6511f7ea55264589773edcdb
Dangermond Escondido 1 Depth to Groundwater 6511f7ea5c0d5f9de3698418
Dangermond Escondido 2 Depth to Groundwater 6511f7ea8081871abec93c0d
Dangermond Escondido 3B Depth to Groundwater 6511f7ea8081871bfbc93c0f
Dangermond Escondido 4 Depth to Groundwater 6511f7ead07087d401d574e5
Dangermond Gaspar 2 Depth 

In [115]:
df = dendra.get_datapoints_from_id_list(measurement_list, begins_at, ends_before)

datastream ID(6511f67d2148db626248533e)  has no data for this time period. Skipping.
datastream ID(6511f7e82148db3a1e485341)  has no data for this time period. Skipping.
datastream ID(6511f7e85526453a3c3edcd7)  has no data for this time period. Skipping.
datastream ID(6511f7e8808187594cc93c0b)  has no data for this time period. Skipping.
datastream ID(6511f7e8d070873fc4d574e2)  has no data for this time period. Skipping.
datastream ID(6511f7e90209692d22a0add0)  has no data for this time period. Skipping.
datastream ID(6511f7e95526459d613edcd9)  has no data for this time period. Skipping.
datastream ID(6511f7e95c0d5fe521698416)  has no data for this time period. Skipping.
datastream ID(6511f7e9f1438dfeb44e11a9)  has no data for this time period. Skipping.
datastream ID(6511f7ea55264589773edcdb)  has no data for this time period. Skipping.
datastream ID(6511f7ea5c0d5f9de3698418)  has no data for this time period. Skipping.
datastream ID(6511f7ea8081871abec93c0d)  has no data for this tim

ValueError: unconverted data remains when parsing with format "%Y-%m-%dT%H:%M:%S": ".000", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.